## Treinando uma rede neural Multilayer Perceptron

Vamos utilizar a rede neural Multilayer Perceptron da biblioteca Sklearn para tentar separar os dados em classes azul e vermelhas.

Impotando bibliotecas:

In [ ]:
#Bibliotecas auxílio
import numpy as np
import matplotlib.pyplot as plt

#Datasets
import sklearn
import sklearn.datasets

#ML e DP
import sklearn.linear_model
from sklearn.neural_network import MLPClassifier

Vamos utilizar algumas funções do arquivo planar_utils para nos auxiliar 

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

In [ ]:
#Função para visualizar os dados aplicados sobre o modelo de rede neural 

def plot_decision_boundary(model, X, y):
    # Set min and max values and give it some padding
    x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
    y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole grid
    Z = model(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(X[0, :], X[1, :], c=y, cmap=plt.cm.Spectral)

# Função para calcular a sigmoid    
def sigmoid(x):
    """
    Compute the sigmoid of x

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

# Função para plotar o gráfico do dataset

def load_planar_dataset():
    np.random.seed(1)
    m = 400 # number of examples
    N = int(m/2) # number of points per class
    D = 2 # dimensionality
    X = np.zeros((m,D)) # data matrix where each row is a single example
    Y = np.zeros((m,1), dtype='uint8') # labels vector (0 for red, 1 for blue)
    a = 4 # maximum ray of the flower

    for j in range(2):
        ix = range(N*j,N*(j+1))
        t = np.linspace(j*3.12,(j+1)*3.12,N) + np.random.randn(N)*0.2 # theta
        r = a*np.sin(4*t) + np.random.randn(N)*0.2 # radius
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        Y[ix] = j

    X = X.T
    Y = Y.T

    return X, Y

Vamos carregar o conjunto de dados no qual trabalharemos. O código abaixo o carregará nas variáveis X e Y

In [ ]:
X, Y = load_planar_dataset()

Y = Y[0] #neste dataset Y tem uma dimensao a mais, vamos remove-la

print(X.shape)
print(Y.shape)

Observe que nossa base de dados contém duas características (X1 e X2) e o rótulo (vermelho:0 e azul:1)

Vamos plotá-lo:

In [ ]:
plt.scatter(X[0, :], X[1, :], c=Y, s=40, cmap=plt.cm.Spectral)

Observe que este problema é bastante complexo para conseguirmos separar os pontos azuis dos vermelhos com apenas uma linha, como faríamos com um modelo linear simples. Apenas como observação, vamos tentar empregar uma regressão logística:

A regressão logística tenta a partir de um conjunto de observações, um modelo que permita a **predição de valores** tomados por uma variável categórica (vermelho 0 ou azul 1), frequentemente binária, a partir de uma série de variáveis explicativas contínuas e/ou binárias (nossos dados de x1 e x2).

Executando o algoritmo de regressão logística:

In [ ]:
clf = sklearn.linear_model.LogisticRegressionCV()
clf.fit(X.T, Y.T)

In [ ]:
plot_decision_boundary(lambda x: clf.predict(x), X, Y)
plt.title("Regressão Logística")

LR_predictions = clf.predict(X.T)
print ('Taxa de acerto da Regressão Logística: %f ' % float(np.mean(LR_predictions == Y[0])))

Para casos como este precisamos de modelos mais complexos, com superfícies de decisões não lineares. Como as Redes Neurais podem ser vistas com o um conjunto de funções não linearmente combinadas, elas nos possibilitam obter superfícies mais complexas

Vamos treinar o modelo Neural abaixo para vermos se obtemos um resultado melhor

Dado os valores preditos, podemos calcular a função de custo por:

$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right) \large \right) \small \tag{2}$

O Scikit-learn nos oferece um pacote para trabalharmos com redes Perceptron, para isso definimos a arquitetura da rede como:

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(4, 1), activation='tanh', random_state=42)

# hidden_layer_sizes -> quantidade de nosssas redes ocultas (4 camadas ocultas + 1 camada oculta)
# activation -> função de ativação
# random state -> garante que os números aleatórios sejam gerados na mesma ordem.

In [ ]:
clf.fit(X.T, Y.T) #realizando o treinamento

In [ ]:
plot_decision_boundary(lambda x: clf.predict(x), X, Y)
plt.title("Rede Neural")

NN_predictions = clf.predict(X.T)
print ('Taxa de acerto da Rede Neural: %f ' % float(np.mean(NN_predictions == Y[0])))

Observe que com este modelo conseguimos construir uma superfície de decisão um pouco "curva" no espaço $R^2$, já que não estamos mais trabalhando com modelos lineares. Com isso aumentamos nossa taxa de acerto.

Vamos tentar modelos mais complexos para observarmos esse comportamento

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(4, 4), activation='tanh', random_state=42)

In [ ]:
clf.fit(X.T, Y.T)


In [ ]:
plot_decision_boundary(lambda x: clf.predict(x), X, Y)
plt.title("Rede Neural")
NN_predictions = clf.predict(X.T)
print ('Taxa de acerto da Rede Neural: %f ' % float(np.mean(NN_predictions == Y[0])))

Um hiper parâmetro muito importante a ser configurado em um NN é o learning_rate. Caso ele seja muito baixo, a rede necessitará de muitas interações para convergir (muitas vezes milhões), o que inviabiliza o projeto. Porém, se ele for muito alto pode haver um "salto" do mínimo da função pelo gradiente, impossibilitando a convergência do modelo.

Vamos testar no nosso exemplo:

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(4, 4), activation='tanh', random_state=42, learning_rate_init=10.0)

In [ ]:
clf.fit(X.T, Y.T)

In [ ]:
plot_decision_boundary(lambda x: clf.predict(x), X, Y)
plt.title("Rede Neural")

NN_predictions = clf.predict(X.T)
print ('Taxa de acerto da Rede Neural: %f ' % float(np.mean(NN_predictions == Y[0])))

Observe que mesmo utilizando um modelo mais complexo, a nossa rede não conseguiu convergir para o mínimo de erro

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(4, 4), activation='tanh', random_state=42, learning_rate_init=0.1) #learning rate muito baixa

In [ ]:
clf.fit(X.T, Y.T)

In [ ]:
plot_decision_boundary(lambda x: clf.predict(x), X, Y)
plt.title("Rede Neural")

NN_predictions = clf.predict(X.T)
print ('Taxa de acerto da Rede Neural: %f ' % float(np.mean(NN_predictions == Y[0])))

O mesmo acontece com uma learning rate muito baixa

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(4, 4), activation='tanh', random_state=42, learning_rate_init=0.001) #learning rate mais adequada, + baixo

In [ ]:
clf.fit(X.T, Y.T)

In [ ]:
plot_decision_boundary(lambda x: clf.predict(x), X, Y)
plt.title("Rede Neural")

NN_predictions = clf.predict(X.T)
print ('Taxa de acerto da Rede Neural: %f ' % float(np.mean(NN_predictions == Y[0])))

Por fim, uma learning rate adequada resulta em um modelo mais preciso

E para prevermos um valor:

In [ ]:
clf.predict([[2.5, 0.75]]) #azul = 1 e vermelho = 0

Assim, nosso ponto $(2.5, 0.75)$ é da classe roxo (ou azul)

In [ ]:
clf.predict_proba([[2.5, 0.75]])

A probabilidade para a classe vermelha é 0.32 e para a roxa 0.68

## Desafio!

Com base nos hiperparâmetros encontrados no link: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html construa uma rede neural que supere a taxa de acerto de 55%.

Vamos realizar uma nova tentativa com outros parâmetros: